In [1]:
import sys
from pathlib import Path
import json
PATH=str((Path().cwd().parent /'src').absolute())
print(PATH)
if PATH not in sys.path:
    sys.path.append(PATH)

/Users/dmarx/proj/chat2obs/src


In [2]:
root = "../data/ingestion/chatgpt/a40ff5f79c1b3edd3c366f0f628fb79170bae83ecf3a1758b5b258c71f843f53-2025-06-05-03-28-15-df2ed357a4e64443bf464446686c9692/"
fpath = Path(root) / "conversations.json"
convs = json.load(fpath.open())

In [5]:
from conversation_tagger import create_default_tagger

tagger = create_default_tagger()
tagged_results = [tagger.tag_conversation(c) for c in convs]

In [6]:
for i, tr in enumerate(tagged_results):
    print(tr.tags)
    if i > 10:
        break   


[Tag('has_web_search', {}), Tag('conversation_length', {'count': 5, 'category': 'medium'}), Tag('conversation_has_web_search', {'exchange_count': 3, 'total_exchanges': 5, 'percentage': 60.0})]
[Tag('has_web_search', {}), Tag('has_reasoning_thoughts', {}), Tag('assistant_has_reasoning', {}), Tag('conversation_length', {'count': 5, 'category': 'medium'}), Tag('conversation_has_web_search', {'exchange_count': 4, 'total_exchanges': 5, 'percentage': 80.0}), Tag('conversation_has_reasoning_thoughts', {'exchange_count': 4, 'total_exchanges': 5, 'percentage': 80.0})]
[Tag('has_web_search', {}), Tag('first_user_has_code_patterns', {}), Tag('has_reasoning_thoughts', {}), Tag('assistant_has_reasoning', {}), Tag('conversation_length', {'count': 21, 'category': 'long'}), Tag('conversation_has_web_search', {'exchange_count': 9, 'total_exchanges': 21, 'percentage': 42.9}), Tag('conversation_has_reasoning_thoughts', {'exchange_count': 13, 'total_exchanges': 21, 'percentage': 61.9})]
[Tag('first_user_h